In [1]:
import pandas as pd

df = pd.read_csv("Agriculture_price_dataset.csv")

In [2]:

# Keep only rows where state = Maharashtra
df = df[df['STATE'] == 'Maharashtra']

df.to_csv("filtered.csv", index=False)

In [18]:
unique_pairs = df[['District Name','Market Name']].drop_duplicates()

for i in unique_pairs.iterrows():
    if "Market" in i[1][1] or "Bazar" in i[1][1]:
        print(i[1][1])

Chandur Bazar
Amrawati(Frui & Veg. Market)
Sangli(Phale, Bhajipura Market)
Varud(Rajura Bazar)
Laxmi Sopan Agriculture Produce Marketing Co Ltd
Shree Salasar Krushi Bazar
Shree Rameshwar Krushi Market
Shree Sairaj Krushi Market
Janata Agri Market (DLS Agro Infrastructure Pvt Lt
Sant Namdev Krushi Bazar,
Premium Krushi Utpanna Bazar


C:\Users\Atharv\AppData\Local\Temp\ipykernel_21468\2051475019.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if "Market" in i[1][1] or "Bazar" in i[1][1]:
C:\Users\Atharv\AppData\Local\Temp\ipykernel_21468\2051475019.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(i[1][1])


In [35]:
df["District Name"] = df["District Name"].str.lower()
df["Market Name"] = df["Market Name"].str.lower()

In [53]:
df2 =  pd.read_csv("cities.csv")
df2["city"] = df2["city"].str.lower()
df2["district"] = df2["district"].str.lower()
df2 = df2.drop_duplicates(subset=["district"])
df2["district"] = df2["district"].replace("amravati", "amarawati")
merged = df.merge(df2, left_on="District Name", right_on="district", how="left")


print(merged)

             STATE District Name                             Market Name  \
0      Maharashtra        nashik                       lasalgaon(niphad)   
1      Maharashtra        satara                                   patan   
2      Maharashtra        nashik                      lasalgaon(vinchur)   
3      Maharashtra    ahmednagar                                pathardi   
4      Maharashtra         dhule  pratap nana mahale khajgi bajar samiti   
...            ...           ...                                     ...   
51811  Maharashtra          pune                            khed(chakan)   
51812  Maharashtra    chandrapur                     chandrapur(ganjwad)   
51813  Maharashtra          pune                           pune(khadiki)   
51814  Maharashtra       jalgaon                                bhusaval   
51815  Maharashtra        nashik                              pimpalgaon   

      Commodity           Variety  Grade  Min_Price  Max_Price  Modal_Price  \
0       

In [55]:
merged = merged.dropna(subset=["city"])
merged.to_csv("joined.csv", index=False)

In [56]:
test = merged.copy()
test["Price Date"] = pd.to_datetime(test["Price Date"], errors="coerce")
test.to_csv("date.csv",  index = True)

In [30]:
for i, roes in unique_pairs.iterrows():
    print(i, roes)

0 District Name               nashik
Market Name      Lasalgaon(Niphad)
lat                       9.904997
lon                       78.10483
Name: 0, dtype: object
1 District Name       satara
Market Name          Patan
lat              18.906836
lon              75.674158
Name: 1, dtype: object
19 District Name                nashik
Market Name      Lasalgaon(Vinchur)
lat                        9.904997
lon                        78.10483
Name: 19, dtype: object
39 District Name    ahmednagar
Market Name        Pathardi
lat               18.906836
lon               75.674158
Name: 39, dtype: object
41 District Name                                     dhule
Market Name      Pratap Nana Mahale Khajgi Bajar Samiti
lat                                           18.906836
lon                                           75.674158
Name: 41, dtype: object
42 District Name       satara
Market Name        Phaltan
lat              18.906836
lon              75.674158
Name: 42, dtype: object
44 Dis

In [57]:
sdf = pd.read_csv("semifinal.csv")
A = sdf.head()

In [59]:
A.to_csv('test.csv')

In [60]:
sem = pd.read_csv("semifinal.csv")
test = sem.head()
test.to_csv("test.csv")

In [10]:
models= dict()

In [5]:
import pandas as pd
import numpy as np

# --- Configuration ---
# Define the target commodity for this time series analysis.
TARGET_COMMODITY = 'Onion'
# Define the number of preceding weeks to use as features
LAG_WINDOW = 4 

def load_data(file_path):
    """Loads the CSV file into a DataFrame."""
    print(f"Loading data from {file_path}...")
    df = pd.read_csv(file_path, index_col=False)
    # Drop the first two unnamed/redundant index columns if they exist
    df = df.iloc[:, 2:]
    return df

def drop_redundant_columns(df):
    """
    Drops columns that are constant, near-constant, or provide redundant
    or non-numerical/non-essential information for this time series model.
    """
    columns_to_drop = [
        'STATE', 'Variety', 'Grade', 'id', 'city', 'std-code', 
        'state', 'gst-state-code', 'iso_3166-2', 'latitude_x', 
        'longitude_x', 'altitude', 'date', 'latitude_y', 'longitude_y', 
        'daylight_duration', 'sunshine_duration',
    ]
    columns_to_drop.extend(['population', 'rank'])
    
    columns_to_drop = list(set(columns_to_drop) & set(df.columns))
    if 'Market Name' in columns_to_drop:
        columns_to_drop.remove('Market Name')
        
    print(f"Dropping {len(columns_to_drop)} columns.")
    df_cleaned = df.drop(columns=columns_to_drop, errors='ignore')
    return df_cleaned

def preprocess_and_aggregate_weekly(df_cleaned, commodity=TARGET_COMMODITY):
    """
    Filters data, converts to datetime, and aggregates to weekly average.
    """
    df_filtered = df_cleaned[df_cleaned['Commodity'] == commodity].copy()
    print(f"\nFiltered data for commodity: {commodity}. Records: {len(df_filtered)}")
    
    if df_filtered.empty:
        print(f"No data found for {commodity}.")
        return None

    df_filtered['Price Date'] = pd.to_datetime(df_filtered['Price Date'])
    df_filtered = df_filtered.sort_values(by=['Market Name', 'Price Date'])
    df_filtered = df_filtered.drop(columns=['Commodity', 'District Name'], errors='ignore')

    # Aggregation Rules
    agg_rules = {
        'Min_Price': 'mean',
        'Max_Price': 'mean',
        'Modal_Price': 'mean',
        'temperature_2m_mean': 'mean',
        'temperature_2m_max': 'max',
        'temperature_2m_min': 'min',
        'wind_speed_10m_max': 'mean',
        'precipitation_sum': 'sum', 
        'precipitation_hours': 'sum',
        # We aggregate weather_code by Mode (most common weather that week)
        'weather_code': lambda x: x.mode()[0] if not x.empty else np.nan
    }
    agg_rules = {col: func for col, func in agg_rules.items() if col in df_filtered.columns}

    def resample_market(market_group):
        market_group = market_group.set_index('Price Date').sort_index()
        return market_group.resample('W').agg(agg_rules).fillna(method='ffill')

    df_weekly = df_filtered.groupby('Market Name').apply(resample_market)
    
    print(f"Data aggregated weekly for {commodity}.")
    return df_weekly

def create_lagged_features(df_weekly, lag_window=LAG_WINDOW):
    """
    Creates lagged features for weather columns and Modal_Price.
    UPDATED: Now includes weather_code as a direct numerical lag (no OHE).
    """
    print(f"\nCreating lagged features (Lag 1 up to Lag {lag_window})...")
    
    # Added 'weather_code' to this list to implement the lag
    features_to_lag = [
        'Modal_Price',
        'temperature_2m_mean', 
        'temperature_2m_max', 
        'temperature_2m_min', 
        'wind_speed_10m_max', 
        'precipitation_sum',
        'precipitation_hours',
        'weather_code'  # <-- NOW INCLUDED AS STANDARD LAG
    ]

    features_to_lag = [f for f in features_to_lag if f in df_weekly.columns]
    df_lagged = df_weekly.copy()
    
    for feature in features_to_lag:
        for lag in range(1, lag_window + 1):
            new_col_name = f'{feature}_Lag{lag}'
            df_lagged[new_col_name] = df_lagged.groupby(level=0)[feature].shift(lag)

    df_lagged = df_lagged.dropna()

    print(f"Finished creating lagged features. Final Shape: {df_lagged.shape}")
    print("Features include simple weather_code lags (e.g., weather_code_Lag1).")
    
    return df_lagged

if __name__ == "__main__":
    file_path = 'semifinal.csv' 
    df = load_data(file_path)
    df_cleaned = drop_redundant_columns(df)
    df_time_series = preprocess_and_aggregate_weekly(df_cleaned, commodity=TARGET_COMMODITY)

    if df_time_series is not None:
        df_final_features = create_lagged_features(df_time_series, lag_window=LAG_WINDOW)
        output_file = f'{TARGET_COMMODITY.lower()}_market_lagged_features.csv'
        df_final_features.to_csv(output_file)
        print(f"\n--- SUCCESS ---")
        print(f"Saved to: {output_file}")
        print("Please re-run model_training.py now.")

Loading data from semifinal.csv...
Dropping 18 columns.

Filtered data for commodity: Onion. Records: 24962


C:\Users\Atharv\AppData\Local\Temp\ipykernel_20184\2203275894.py:72: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  return market_group.resample('W').agg(agg_rules).fillna(method='ffill')
C:\Users\Atharv\AppData\Local\Temp\ipykernel_20184\2203275894.py:74: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_weekly = df_filtered.groupby('Market Name').apply(resample_market)


Data aggregated weekly for Onion.

Creating lagged features (Lag 1 up to Lag 4)...
Finished creating lagged features. Final Shape: (7973, 42)
Features include simple weather_code lags (e.g., weather_code_Lag1).

--- SUCCESS ---
Saved to: onion_market_lagged_features.csv
Please re-run model_training.py now.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, make_scorer
from sklearn.preprocessing import StandardScaler
import pickle 

# --- Configuration ---
INPUT_FILE = 'onion_market_lagged_features.csv'  # Make sure this matches your data prep output
TARGET_COLUMN = 'Modal_Price'
TEST_SPLIT_RATIO = 0.2 
N_SPLITS = 5 
COMMODITY_NAME = 'Onion'

# --- GPU Configuration ---
GPU_ENABLED_PARAMS = {
    'tree_method': 'hist', 
    'device': 'cuda'
}

def load_and_prepare_data(file_path):
    try:
        # Ensure we parse dates correctly
        df = pd.read_csv(file_path, index_col=[0, 1], parse_dates=True)
        print(f"Loaded data. Shape: {df.shape}")
    except FileNotFoundError:
        print(f"Error: {file_path} not found.")
        return None, None, None

    y = df[TARGET_COLUMN]
    # Drop current week targets so we don't cheat
    X = df.drop(columns=['Min_Price', 'Max_Price', TARGET_COLUMN], errors='ignore')
    return df, X, y

def split_and_scale_data(X, y):
    # Use time-based split, not random
    split_index = int(len(X) * (1 - TEST_SPLIT_RATIO))
    
    X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
    y_train, y_test = y.iloc[:split_index], y.iloc[split_index:]

    print(f"Train size: {len(X_train)}, Test size: {len(X_test)}")
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
    X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.columns)
    
    return X_train_scaled_df, X_test_scaled_df, y_train, y_test, scaler

def tune_model_with_tscv(X_train, y_train):
    print("\n--- Tuning Model (Conservative Mode) ---")

    # FIXED: Added regularization to prevent extreme price predictions
    param_grid = {
        'n_estimators': [300, 500],
        'max_depth': [3, 5],       # Lower depth = less overfitting to spikes
        'learning_rate': [0.01, 0.05],
        'subsample': [0.7, 0.8],
        'colsample_bytree': [0.7, 0.8],
        'reg_alpha': [0.1, 1.0],   # L1 Regularization (conservatism)
        'reg_lambda': [1.0, 2.0]   # L2 Regularization (conservatism)
    }

    xgb = XGBRegressor(
        objective='reg:squarederror', 
        random_state=42, 
        n_jobs=-1,
        **GPU_ENABLED_PARAMS
    )

    tscv = TimeSeriesSplit(n_splits=N_SPLITS)
    
    random_search = RandomizedSearchCV(
        estimator=xgb,
        param_distributions=param_grid,
        scoring='neg_mean_absolute_error', # Optimize for MAE (Dollar amount) not MSE
        cv=tscv,
        n_iter=15, 
        verbose=1,
        random_state=42,
        n_jobs=-1
    )

    random_search.fit(X_train, y_train)
    print(f"Best MAE params: {random_search.best_params_}")
    return random_search.best_estimator_

def final_evaluation(model, X_test, y_test):
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f"\n--- Final Test Set Metrics ---")
    print(f"MAE: {mae:.2f}")
    print(f"R2: {r2:.4f}")
    
    # Check for bias
    avg_actual = np.mean(y_test)
    avg_pred = np.mean(y_pred)
    print(f"Avg Actual: {avg_actual:.2f} | Avg Predicted: {avg_pred:.2f}")
    if avg_pred > avg_actual * 1.1:
        print("WARNING: Model is systematically over-predicting.")

def save_model_artifacts(model, scaler):
    with open('Onion.pkl', 'wb') as file:
        pickle.dump(model, file)
    with open('scaler.pkl', 'wb') as file:
        pickle.dump(scaler, file)
    print("Artifacts saved.")

if __name__ == "__main__":
    df_data, X, y = load_and_prepare_data(INPUT_FILE)
    if X is not None:
        X_train, X_test, y_train, y_test, scaler = split_and_scale_data(X, y)
        best_xgb_model = tune_model_with_tscv(X_train, y_train)
        final_evaluation(best_xgb_model, X_test, y_test)
        save_model_artifacts(best_xgb_model, scaler,"Onion.pkl","o_scaler.pkl")

C:\Users\Atharv\AppData\Local\Temp\ipykernel_20184\802727568.py:25: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(file_path, index_col=[0, 1], parse_dates=True)


Loaded data. Shape: (7973, 42)
Train size: 6378, Test size: 1595

--- Tuning Model (Conservative Mode) ---
Fitting 5 folds for each of 15 candidates, totalling 75 fits
Best MAE params: {'subsample': 0.7, 'reg_lambda': 2.0, 'reg_alpha': 0.1, 'n_estimators': 300, 'max_depth': 3, 'learning_rate': 0.05, 'colsample_bytree': 0.8}

--- Final Test Set Metrics ---
MAE: 201.30
R2: 0.9326
Avg Actual: 2333.72 | Avg Predicted: 2325.69


TypeError: save_model_artifacts() takes 2 positional arguments but 4 were given

In [9]:
save_model_artifacts(best_xgb_model, scaler)

Artifacts saved.
